In [1]:
#Copyright 2020 Vraj Shah, Arun Kumar
#
#Licensed under the Apache License, Version 2.0 (the "License");
#you may not use this file except in compliance with the License.
#You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
#Unless required by applicable law or agreed to in writing, software
#distributed under the License is distributed on an "AS IS" BASIS,
#WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#See the License for the specific language governing permissions and
#limitations under the License.

import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import numpy as np

from Load_Predictions import *
from downstream_models import *
from Featurize import *
from Train_Test_Random_Forest import *

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Andrew\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
dict_label_true = {
    'numeric': 0,
    'categorical': 1,
    'datetime': 2,
    'sentence': 3,
    'url': 4,
    'embedded-number': 5,
    'list': 6,
    'not-generalizable': 7,
    'context-specific': 8
}

# y_true = [dict_label_true[str(i)] for i in y_true]

In [3]:
table_column_names = ['Record_id', 'Attribute_name', 'y_act', 'total_vals', 'num_nans',
           '%_nans', 'num_of_dist_val', '%_dist_val', 'mean', 'std_dev', 'min_val',
           'max_val', 'has_delimiters', 'has_url', 'has_email', 'has_date', 'mean_word_count',
           'std_dev_word_count', 'mean_stopword_total', 'stdev_stopword_total',
           'mean_char_count', 'stdev_char_count', 'mean_whitespace_count',
           'stdev_whitespace_count', 'mean_delim_count', 'stdev_delim_count',
           'is_list', 'is_long_sentence', 
           'sample_1', 'sample_2', 'sample_3', 'sample_4', 'sample_5',
           'sample_6', 'sample_7', 'sample_8', 'sample_9', 'sample_10',]

In [4]:
removed_sample_statistics = ['std_dev_word_count', 'mean_stopword_total',
       'stdev_stopword_total', 'mean_char_count', 'stdev_char_count',
       'mean_whitespace_count', 'stdev_whitespace_count', 'mean_delim_count',
       'stdev_delim_count']

In [5]:
sample_size_list = [1,2,3,4,5,10]
rand_seed = 100
results = []

final_results = []

for sample_size in sample_size_list:
    
    RF_results = []
    
    xtrain = pd.read_csv(f"Sampled-Benchmark-Labeled-Data/{sample_size}_sample_data_train.csv")
    xtest = pd.read_csv(f"Sampled-Benchmark-Labeled-Data/{sample_size}_sample_data_test.csv")
    

    y_true, RF_results = Train_Test_Random_Forest(xtrain, xtest, truncated=True)
    
    print(y_true)
    print(RF_results)
    
    results = pd.DataFrame(
    {'actual': y_true['y_act'].values,
     'predicted': RF_results
    })
    
    cmat = confusion_matrix(results['actual'], results['predicted'])
    accuracies = []

    for key, value in dict_label_true.items(): 
        TP=0
        FP=0
        TN=0
        FN=0
        for i in range(0,9):
            for j in range(0,9):
                total_actual = sum(cmat[i])
                if (i == value & i == j):
                    TP = cmat[i][j]
                    FP = total_actual-TP
                if i != value:
                    TN += cmat[i][j]
                if (i != value) & (j == value):
                    FN += cmat[i][j]

        print(TP, FP, TN, FN)
        accuracy = (TP+(TN-FN))/(TP+FP+FN+(TN-FN))
        print(accuracy)
        accuracies.append(accuracy)
        
    class_accuracy = list(zip(dict_label_true.keys(), accuracies))
    class_accuracy = pd.DataFrame(class_accuracy).set_index(0).T
    class_accuracy.index = ['accuracy']
    
    classification_report_df = pd.DataFrame(classification_report(results['actual'], results['predicted'], target_names=dict_label_true.keys(), output_dict=True))
    classification_report_df = classification_report_df.loc[['precision', 'recall', 'f1-score']].iloc[: , :-3]
    
    result_df = class_accuracy.append(classification_report_df)
    final_results.append([result_df, cmat])

C:\Users\Andrew\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


[n_estimator: 5, max_depth: 5, accuracy: 0.5122873345935728]
[n_estimator: 5, max_depth: 10, accuracy: 0.5834908632640201]
[n_estimator: 5, max_depth: 25, accuracy: 0.7403906742281033]
[n_estimator: 5, max_depth: 50, accuracy: 0.7908002520478891]
[n_estimator: 5, max_depth: 100, accuracy: 0.7958412098298677]
[n_estimator: 5, max_depth: 250, accuracy: 0.7952110901071203]
[n_estimator: 25, max_depth: 5, accuracy: 0.5293005671077504]
[n_estimator: 25, max_depth: 10, accuracy: 0.6868304977945809]
[n_estimator: 25, max_depth: 25, accuracy: 0.7914303717706365]
[n_estimator: 25, max_depth: 50, accuracy: 0.8185255198487713]
[n_estimator: 25, max_depth: 100, accuracy: 0.8229363579080026]
[n_estimator: 25, max_depth: 250, accuracy: 0.8298676748582231]
[n_estimator: 50, max_depth: 5, accuracy: 0.5122873345935728]
[n_estimator: 50, max_depth: 10, accuracy: 0.6906112161310649]
[n_estimator: 50, max_depth: 25, accuracy: 0.8034026465028355]
[n_estimator: 50, max_depth: 50, accuracy: 0.822936357908002

[n_estimator: 50, max_depth: 25, accuracy: 0.7909319899244333]
[n_estimator: 50, max_depth: 50, accuracy: 0.8243073047858942]
[n_estimator: 50, max_depth: 100, accuracy: 0.8350125944584383]
[n_estimator: 50, max_depth: 250, accuracy: 0.8343828715365239]
[n_estimator: 75, max_depth: 5, accuracy: 0.5906801007556675]
[n_estimator: 75, max_depth: 10, accuracy: 0.6630982367758187]
[n_estimator: 75, max_depth: 25, accuracy: 0.7940806045340051]
[n_estimator: 75, max_depth: 50, accuracy: 0.8243073047858942]
[n_estimator: 75, max_depth: 100, accuracy: 0.8381612090680101]
[n_estimator: 75, max_depth: 250, accuracy: 0.8394206549118388]
[n_estimator: 100, max_depth: 5, accuracy: 0.5661209068010076]
[n_estimator: 100, max_depth: 10, accuracy: 0.6706549118387909]
[n_estimator: 100, max_depth: 25, accuracy: 0.7965994962216625]
[n_estimator: 100, max_depth: 50, accuracy: 0.8287153652392947]
[n_estimator: 100, max_depth: 100, accuracy: 0.836272040302267]
[n_estimator: 100, max_depth: 250, accuracy: 0.8

C:\Users\Andrew\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


[n_estimator: 5, max_depth: 5, accuracy: 0.448503937007874]
[n_estimator: 5, max_depth: 10, accuracy: 0.6]
[n_estimator: 5, max_depth: 25, accuracy: 0.7946456692913386]
[n_estimator: 5, max_depth: 50, accuracy: 0.8560629921259842]
[n_estimator: 5, max_depth: 100, accuracy: 0.8503937007874016]
[n_estimator: 5, max_depth: 250, accuracy: 0.8614173228346457]
[n_estimator: 25, max_depth: 5, accuracy: 0.5675590551181102]
[n_estimator: 25, max_depth: 10, accuracy: 0.6755905511811023]
[n_estimator: 25, max_depth: 25, accuracy: 0.8277165354330709]
[n_estimator: 25, max_depth: 50, accuracy: 0.8856692913385826]
[n_estimator: 25, max_depth: 100, accuracy: 0.8944881889763779]
[n_estimator: 25, max_depth: 250, accuracy: 0.9007874015748032]
[n_estimator: 50, max_depth: 5, accuracy: 0.5637795275590551]
[n_estimator: 50, max_depth: 10, accuracy: 0.6831496062992126]
[n_estimator: 50, max_depth: 25, accuracy: 0.8277165354330709]
[n_estimator: 50, max_depth: 50, accuracy: 0.8853543307086614]
[n_estimator:

...

[n_estimator: 5, max_depth: 5, accuracy: 0.5840473790322581]
[n_estimator: 5, max_depth: 10, accuracy: 0.6446572580645161]
[n_estimator: 5, max_depth: 25, accuracy: 0.8181703629032258]
[n_estimator: 5, max_depth: 50, accuracy: 0.9213709677419355]
[n_estimator: 5, max_depth: 100, accuracy: 0.9319556451612904]
[n_estimator: 5, max_depth: 250, accuracy: 0.9291834677419355]
[n_estimator: 25, max_depth: 5, accuracy: 0.5995463709677419]
[n_estimator: 25, max_depth: 10, accuracy: 0.6731350806451613]
[n_estimator: 25, max_depth: 25, accuracy: 0.8608870967741935]
[n_estimator: 25, max_depth: 50, accuracy: 0.9340977822580645]
[n_estimator: 25, max_depth: 100, accuracy: 0.9488407258064516]
[n_estimator: 25, max_depth: 250, accuracy: 0.9501008064516129]
[n_estimator: 50, max_depth: 5, accuracy: 0.5811491935483871]
[n_estimator: 50, max_depth: 10, accuracy: 0.6950604838709677]
[n_estimator: 50, max_depth: 25, accuracy: 0.8623991935483871]
[n_estimator: 50, max_depth: 50, accuracy: 0.936491935483871

[n_estimator: 50, max_depth: 25, accuracy: 0.8694556451612904]
[n_estimator: 50, max_depth: 50, accuracy: 0.9351058467741935]
[n_estimator: 50, max_depth: 100, accuracy: 0.9474546370967742]
[n_estimator: 50, max_depth: 250, accuracy: 0.9478326612903226]
[n_estimator: 75, max_depth: 5, accuracy: 0.6049647177419355]
[n_estimator: 75, max_depth: 10, accuracy: 0.6873739919354839]
[n_estimator: 75, max_depth: 25, accuracy: 0.8712197580645161]
[n_estimator: 75, max_depth: 50, accuracy: 0.9369959677419355]
[n_estimator: 75, max_depth: 100, accuracy: 0.9478326612903226]
[n_estimator: 75, max_depth: 250, accuracy: 0.9493447580645161]
[n_estimator: 100, max_depth: 5, accuracy: 0.6096270161290323]
[n_estimator: 100, max_depth: 10, accuracy: 0.6896421370967742]
[n_estimator: 100, max_depth: 25, accuracy: 0.8727318548387096]
[n_estimator: 100, max_depth: 50, accuracy: 0.9368699596774194]
[n_estimator: 100, max_depth: 100, accuracy: 0.9484627016129032]
[n_estimator: 100, max_depth: 250, accuracy: 0.

C:\Users\Andrew\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


[n_estimator: 5, max_depth: 5, accuracy: 0.59294265910523]
[n_estimator: 5, max_depth: 10, accuracy: 0.6742281033396346]
[n_estimator: 5, max_depth: 25, accuracy: 0.7933207309388783]
[n_estimator: 5, max_depth: 50, accuracy: 0.8223062381852552]
[n_estimator: 5, max_depth: 100, accuracy: 0.8267170762444864]
[n_estimator: 5, max_depth: 250, accuracy: 0.833648393194707]
[n_estimator: 25, max_depth: 5, accuracy: 0.6244486452425961]
[n_estimator: 25, max_depth: 10, accuracy: 0.7246376811594203]
[n_estimator: 25, max_depth: 25, accuracy: 0.8399495904221802]
[n_estimator: 25, max_depth: 50, accuracy: 0.8632640201638311]
[n_estimator: 25, max_depth: 100, accuracy: 0.872715816005041]
[n_estimator: 25, max_depth: 250, accuracy: 0.8676748582230623]
[n_estimator: 50, max_depth: 5, accuracy: 0.6156269691241336]
[n_estimator: 50, max_depth: 10, accuracy: 0.7340894770006301]
[n_estimator: 50, max_depth: 25, accuracy: 0.8292375551354757]
[n_estimator: 50, max_depth: 50, accuracy: 0.8645242596093258]
[

[n_estimator: 50, max_depth: 25, accuracy: 0.8261964735516373]
[n_estimator: 50, max_depth: 50, accuracy: 0.8595717884130982]
[n_estimator: 50, max_depth: 100, accuracy: 0.8589420654911839]
[n_estimator: 50, max_depth: 250, accuracy: 0.8589420654911839]
[n_estimator: 75, max_depth: 5, accuracy: 0.6190176322418136]
[n_estimator: 75, max_depth: 10, accuracy: 0.7304785894206549]
[n_estimator: 75, max_depth: 25, accuracy: 0.820528967254408]
[n_estimator: 75, max_depth: 50, accuracy: 0.8570528967254408]
[n_estimator: 75, max_depth: 100, accuracy: 0.860831234256927]
[n_estimator: 75, max_depth: 250, accuracy: 0.8602015113350125]
[n_estimator: 100, max_depth: 5, accuracy: 0.6064231738035264]
[n_estimator: 100, max_depth: 10, accuracy: 0.7311083123425692]
[n_estimator: 100, max_depth: 25, accuracy: 0.8249370277078085]
[n_estimator: 100, max_depth: 50, accuracy: 0.8551637279596978]
[n_estimator: 100, max_depth: 100, accuracy: 0.8602015113350125]
[n_estimator: 100, max_depth: 250, accuracy: 0.86

In [6]:
sample_size_results_list = []

for i in range(len(sample_size_list)):
    results_df = final_results[i][0]
    
    results_df = pd.DataFrame(results_df.T.stack()).rename({0: f"Sample Size: {sample_size_list[i]}"}, axis=1)
    results_df.index = results_df.index.set_names(['Feature Type', 'Metric'])
    
    sample_size_results_list.append(results_df)

In [7]:
benchmark_df = pd.concat(sample_size_results_list, axis=1)
benchmark_df

Sameple Size: 1  Sameple Size: 2  \
Feature Type      Metric                                        
numeric           accuracy          0.951134         0.957179   
                  precision         0.906667         0.919703   
                  recall            0.961810         0.963932   
                  f1-score          0.933425         0.941298   
categorical       accuracy          0.911839         0.922418   
                  precision         0.773256         0.804217   
                  recall            0.873085         0.876368   
                  f1-score          0.820144         0.838743   
datetime          accuracy          0.992947         0.995466   
                  precision         0.956835         0.958333   
                  recall            0.943262         0.978723   
                  f1-score          0.950000         0.968421   
sentence          accuracy          0.977834         0.981612   
                  precision         0.842857         0.853503   
                  recall            0.641304         0.728261   
                  f1-score          0.728395         0.785924   
url               accuracy          0.996474         0.996725   
                  precision         0.931034         0.947368   
                  recall            0.843750         0.843750   
                  f1-score          0.885246         0.892562   
embedded-number   accuracy          0.984383         0.985139   
                  precision         0.840000         0.835749   
                  recall            0.848485         0.873737   
                  f1-score          0.844221         0.854321   
list              accuracy          0.985390         0.986146   
                  precision         0.818182         0.864198   
                  recall            0.631579         0.614035   
                  f1-score          0.712871         0.717949   
not-generalizable accuracy          0.927456         0.931234   
                  precision         0.689840         0.698734   
                  recall            0.600000         0.641860   
                  f1-score          0.641791         0.669091   
context-specific  accuracy          0.944081         0.945844   
                  precision         0.746667         0.752443   
                  recall            0.605405         0.624324   
                  f1-score          0.668657         0.682422   

                             Sameple Size: 3  Sameple Size: 4  \
Feature Type      Metric                                        
numeric           accuracy          0.961713         0.962720   
                  precision         0.925393         0.926550   
                  recall            0.970769         0.972419   
                  f1-score          0.947538         0.948930   
categorical       accuracy          0.923090         0.923426   
                  precision         0.799344         0.805611   
                  recall            0.889132         0.879650   
                  f1-score          0.841851         0.841004   
datetime          accuracy          0.996641         0.996474   
                  precision         0.967517         0.971831   
                  recall            0.985816         0.978723   
                  f1-score          0.976581         0.975265   
sentence          accuracy          0.981360         0.981486   
                  precision         0.857143         0.833837   
                  recall            0.717391         0.750000   
                  f1-score          0.781065         0.789700   
url               accuracy          0.998153         0.998363   
                  precision         0.956989         0.960000   
                  recall            0.927083         0.937500   
                  f1-score          0.941799         0.948617   
embedded-number   accuracy          0.988413         0.986524   
                  precision         0.882550         0.858561  

In [8]:
benchmark_df.to_latex('Retrained Truncated Sample Size Benchmark.tex')

In [11]:
benchmark_df.to_csv('Retrained Truncated Sample Size Benchmark.csv')

In [10]:
for i in range(len(sample_size_list)):
    cmat_df = final_results[i][1]
    accuracy = np.trace(cmat_df) / np.sum(cmat_df)
    print(accuracy)

0.8357682619647355
0.8508816120906801
0.8609571788413098
0.8604534005037784
0.8615617128463476
0.8634760705289672
